In [2]:
from pyboy.core.mb import Motherboard
from pyboy import PyBoyMemoryView

In [3]:
# Initializing Motherboard with gamerom: 
# red.gbc, 
# bootrom: None, 
# color_palette: (16777215, 10066329, 5592405, 0), 
# cgb_color_palette: ((16777215, 8126257, 25541, 0), 
# (16777215, 16745604, 9714234, 0), 
# (16777215, 16745604, 9714234, 0)), 
# sound: False, 
# sound_emulated: False, 
# cgb: None, 
# randomize: False

In [4]:
class RamReader:
    def __init__(self, gbc):
        mb = Motherboard(
            "red.gbc",
            None,
            (16777215, 10066329, 5592405, 0),
            ((16777215, 8126257, 25541, 0), 
                (16777215, 16745604, 9714234, 0), 
                (16777215, 16745604, 9714234, 0)
            ),
            False,
            False,
            None,
            False,
        )
        
        self.memory = PyBoyMemoryView(mb)

In [5]:
pyboy = RamReader("red.gbc")

the theory is that 0xA598 is the address of the player name in sram
```
0x0000 - 0x3FFF : ROM Bank 0
0x4000 - 0x7FFF : ROM Bank 1-N (switchable)
0x8000 - 0x9FFF : Video RAM (VRAM)
0xA000 - 0xBFFF : External/Cartridge RAM (SRAM)
0xC000 - 0xDFFF : Work RAM (WRAM)
0xE000 - 0xFDFF : Echo RAM (mirror of 0xC000-0xDDFF)
0xFE00 - 0xFE9F : Sprite Attribute Memory (OAM)
0xFEA0 - 0xFEFF : Not Usable
0xFF00 - 0xFF7F : I/O Registers
0xFF80 - 0xFFFE : High RAM (HRAM)
0xFFFF         : Interrupt Enable Register
```
[here](https://gbdev.io/pandocs/Memory_Map.html) is a deeper dive

[here](https://bulbapedia.bulbagarden.net/wiki/Character_encoding_(Generation_I)#English) is the character encoding business

In [46]:
CHAR_MAP = {
    # Control characters (row 6)
    0x60: 'A', 0x61: 'B', 0x62: 'C', 0x63: 'D', 0x64: 'E', 0x65: 'F',
    0x66: 'G', 0x67: 'H', 0x68: 'I', 0x69: 'V', 0x6A: 'S', 0x6B: 'L',
    0x6C: 'M', 0x6D: ':', 0x6E: 'い', 0x6F: 'う',

    # Row 7 (special characters)
    0x70: ''', 0x71: ''', 0x72: 'u', 0x73: '"', 0x74: '・',
    0x75: '...', 0x76: 'あ', 0x77: 'え', 0x78: 'お', 0x79: '┌',
    0x7A: '=', 0x7B: '┐', 0x7C: '|', 0x7D: '└', 0x7E: '┘',
    0x7F: 'SP',

    # Upper case letters (row 8)
    0x80: 'A', 0x81: 'B', 0x82: 'C', 0x83: 'D', 0x84: 'E', 0x85: 'F',
    0x86: 'G', 0x87: 'H', 0x88: 'I', 0x89: 'J', 0x8A: 'K', 0x8B: 'L',
    0x8C: 'M', 0x8D: 'N', 0x8E: 'O', 0x8F: 'P',

    # More upper case and symbols (row 9)
    0x90: 'Q', 0x91: 'R', 0x92: 'S', 0x93: 'T', 0x94: 'U', 0x95: 'V',
    0x96: 'W', 0x97: 'X', 0x98: 'Y', 0x99: 'Z', 0x9A: '(', 0x9B: ')',
    0x9C: ':', 0x9D: ';', 0x9E: '[', 0x9F: ']',

    # Lower case letters a-p (row A)
    0xA0: 'a', 0xA1: 'b', 0xA2: 'c', 0xA3: 'd', 0xA4: 'e', 0xA5: 'f',
    0xA6: 'g', 0xA7: 'h', 0xA8: 'i', 0xA9: 'j', 0xAA: 'k', 0xAB: 'l',
    0xAC: 'm', 0xAD: 'n', 0xAE: 'o', 0xAF: 'p',

    # Lower case letters q-z and symbols (row B)
    0xB0: 'q', 0xB1: 'r', 0xB2: 's', 0xB3: 't', 0xB4: 'u', 0xB5: 'v',
    0xB6: 'w', 0xB7: 'x', 0xB8: 'y', 0xB9: 'z', 0xBA: 'é', 0xBB: '\'d',
    0xBC: 'l', 0xBD: 's', 0xBE: 't', 0xBF: 'v',

    # Row E (symbols and special characters)
    0xE0: '\'', 0xE1: 'P', 0xE2: 'M', 0xE3: '-', 0xE4: 'r', 0xE5: 'm',
    0xE6: '?', 0xE7: '!', 0xE8: '.', 0xE9: 'ア', 0xEA: 'ウ', 0xEB: 'エ',
    0xEC: '▷', 0xED: '▶', 0xEE: '▼', 0xEF: '♂',

    # Row F (numbers and symbols)
    0xF0: '¥', 0xF1: '×', 0xF2: '.', 0xF3: '/', 0xF4: ',', 0xF5: '♀',
    0xF6: '0', 0xF7: '1', 0xF8: '2', 0xF9: '3', 0xFA: '4', 0xFB: '5',
    0xFC: '6', 0xFD: '7', 0xFE: '8', 0xFF: '9',

    # Special control characters
    0x50: '@'  # String terminator
}

def decode_s(encoded_str: list[int]):
    res = ''
    for encoded_c in encoded_str:
        if encoded_c == 0x50:
            break
        res += CHAR_MAP[encoded_c]
    return res

In [47]:
NAME_LENGTH = 11 # bytes
PLAYER_NAME_START = 0xA598 # from our symbol table
name = pyboy.memory[1, PLAYER_NAME_START:PLAYER_NAME_START + NAME_LENGTH] # important that we from rom bank 1
decode_s(name)

'RED'

cool it works! we now know how to read from SRAM, now lets try loading in the game state, and then try editing it and creating a cool little modding interface so we can intialize the game from different positions and run it

the heirarchy of the game data looks something like this

```
sGameData::
sPlayerName::  ds NAME_LENGTH
sMainData::    ds wMainDataEnd - wMainDataStart
sSpriteData::  ds wSpriteDataEnd - wSpriteDataStart
sPartyData::   ds wPartyDataEnd - wPartyDataStart
sCurBoxData::  ds wBoxDataEnd - wBoxDataStart
sTileAnimations:: db
sGameDataEnd::
```

so ideally we want to create a datastructure which can display in string/ ints what the values of all these things are, and then can update all these values as well, this is going to be a multi-stage process, lets first start by creating a little thing which can just read this

we kind of want it to be recursive, something which describes the layout of a thing, then the layout of its components, sounds kinda like this can be a nested dict or maybe a tree?

In [51]:
MAX_CHARACTERS=11
GAME_DATA_LAYOUT = {
    "sGameData": {
        "start_addr": 0xa598,
        "size": 3979,
        "rom_bank": 1,
        "values": [
            "sPlayerName",
            "sMainData",
            "sSpriteData",
            "sPartyData",
            "sCurBoxData",
            "sTileAnimations",
            "sGameDataEnd"
        ],
        "type": "struct"
    },
    "sPlayerName": {
        "start_addr": 0xa598,
        "size": 0xa598 + MAX_CHARACTERS,
        "rom_bank": 1,
        "type": "string"
    },
    "sMainData": {
        "start_addr": 0xa5a3,
        "rom_bank": 1,
        "values": [
            "wPokedexOwned",
            "wPokedexSeen",
            "wNumBagItems",
            "wBagItems",
            "wPlayerMoney",
            "wRivalName",
            "WOptions",
            "wObtainedBadges",
            "wUnusedObtainedBadges",
            "wLetterPrintingDelayFlags",
            "wPlayerID",
            "wMapMusicSoundID",
            "wMapMusicROMBank",
            "wMapPalOffset",
            "wCurMap",
            "wCurrentTileBlockMapViewPointer",
            "wYCoord",
            "wXCoord",
            "wYBlockCoord",
            "wXBlockCoord",
            "wLastMap",
            "wUnusedLastMapWidth",
            "wCurMapHeader",
            "wCurMapTileset",
            "wCurMapHeight",
            "wCurMapWidth",
            "wCurMapDataPtr",
            "wCurMapTextPtr",
            "wCurMapScriptPtr",
            "wCurMapConnections",
            "wNorthConnectionHeader",
            "wSouthConnectionHeader",
            "wWestConnectionHeader",
            "wEastConnectionHeader",
            "wSpriteSet",
            "wSpriteSetID",
            "wObjectDataPointerTemp",
            "
        ]
    },
    "sSpriteData": {
        "start_addr": 0xad2c,
        "rom_bank": 1
    },
    "sPartyData": {
        "start_addr": 0xaf2c,
        "rom_bank": 1
    },
    "sCurBoxData": {
        "start_addr": 0xb0c0,
        "rom_bank": 1
    },
    "sTileAnimations": {
        "start_addr": 0xb522,
        "rom_bank": 1,
        "size": 1
    },
}

In [45]:
MAIN_DATA_START = 0xA598
DATA_LEN = 0xda80 - 0xd2f7 # 1929
main_data = pyboy.memory[1, MAIN_DATA_START:MAIN_DATA_START + DATA_LEN]